# [Exploration_05] 🤖 한국어 데이터로 챗봇 만들기

---

<br>  

영어로 만들었던 챗봇을 한국어 데이터로 바꿔서 훈련시켜보자.  

<br>  
<br>  
<br>  
<br>  
<br>  
<br>  

## 🔶Step 1. 데이터 수집하기

---  

한국어 챗봇 데이터는 송영숙님이 공개한 챗봇 데이터를 사용

In [103]:
$ mkdir -p ~/aiffel/transformer_chatbot/data/
$ ln -s ~/data/* ~/aiffel/transformer_chatbot/data/

<br>  
<br>  

## 🔶Step 2. 데이터 전처리하기

---  


In [2]:
# 라이브러리 확인하기

import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

2.6.0


In [3]:
path_to_chatbot_data = ('/aiffel/aiffel/transformer_chatbot/data/ChatbotData .csv')
train_data = pd.read_csv(path_to_chatbot_data)

In [4]:
# Chatbot 샘플의 개수 확인
print(len(train_data))

11823


In [5]:
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [6]:
questions = []
for sentence in train_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [7]:
answers = []
for sentence in train_data['A']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

- `([?.!,])`: 괄호 안의 문자들(?, ., !, ,)은 이것을 하위 그룹으로 묶는다. 이는 매칭된 구두점을 나중에 참조하기 위한 것이다.
- `r" \1 "`: 이 부분은 찾은 구두점을 가리킨다. `\1` 은 첫 번째 하위 그룹을 의미하며, 찾은 구두점을 나타낸다. 여기에는 앞뒤로 공백을 추가하여 구두점 주변에 공백을 만드는 역할을 한다.
- `re.sub(r"([?.!,])", r" \1 ", sentence)`: 이 부분은 주어진 문장에서 정규 표현식 `([?.!,])`에 매치되는 부분(즉, 구두점)을 찾아서 해당 부분 앞뒤에 공백을 추가한다.   

**이를 통해 구두점 주변에 공백을 추가하여 문장을 좀 더 토큰화하기 쉽게 만들 수 있다.**  

<br>  
<br>  

In [8]:
# 띄어쓰기를 통해 분리되었는지 확인
print(questions[:3])
print(answers[:3])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .']


<br>  

##  🔶Step 3. SubwordTextEncoder 사용하기

---  

한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있다. 하지만 여기서는 형태소 분석기가 아닌 내부 단어 토크나이저인 `SubwordTextEncoder`를 그대로 사용하겠다.  

In [9]:
#서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합(Vocabulary) 생성
tokenizer=tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions+answers,target_vocab_size=2**13)

In [10]:
#시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN,END_TOKEN=[tokenizer.vocab_size],[tokenizer.vocab_size+1]

#시작 토큰과 종료 토큰을 고려햐여 단어 집합의 크기를 +2
VOCAB_SIZE=tokenizer.vocab_size+2

print('시작 토큰 번호: ',START_TOKEN)
print('종료 토큰 번호: ',END_TOKEN)
print('단어 집합의 크기: ',VOCAB_SIZE)

시작 토큰 번호:  [8178]
종료 토큰 번호:  [8179]
단어 집합의 크기:  8180


In [12]:
# 최대 길이를 40으로 정의
MAX_LENGTH=40

# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs,outputs):
    tokenized_inputs,tokenized_outputs=[],[]
    
    for (sentence1,sentence2) in zip(inputs,outputs):
        # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
        sentence1=START_TOKEN+tokenizer.encode(sentence1)+END_TOKEN
        sentence2=START_TOKEN+tokenizer.encode(sentence2)+END_TOKEN
        
        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)
        
    # 최대 길이 40으로 모든 데이터셋을 패딩
    tokenized_inputs=tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs,maxlen=MAX_LENGTH,padding='post')
    tokenized_outputs=tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs,maxlen=MAX_LENGTH,padding='post')

    return tokenized_inputs,tokenized_outputs

In [13]:
questions,answers=tokenize_and_filter(questions,answers)

In [14]:
print('질문 데이터의 크기(shape): ',questions.shape)
print('답변 데이터의 크기(shape): ',answers.shape)

질문 데이터의 크기(shape):  (11823, 40)
답변 데이터의 크기(shape):  (11823, 40)


<br>  

질문과 답변의 쌍을 **tf.data.Dataset API**의 입력으로 사용하여 파이프라인을 구성합니다. 이때, 교사 강요를 위해서 `answers[:, :-1]`를 디코더의 입력값, `answers[:, 1:]`를 디코더의 레이블로 사용합니다. 

<br>  

In [15]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제 값 시퀀스를 구성한다.
BATCH_SIZE=64
BUFFER_SIZE=20000

# 디코더는 이전의 target을 다음의 input으로 사용
# 이에 따라 outputs에서는 START_TOKEN을 제거
dataset=tf.data.Dataset.from_tensor_slices((
    {
        'inputs':questions,
        'dec_inputs':answers[:,:-1] #디코더의 입력. 마지막 패딩 토큰이 제거
    },
    {
      'outputs':answers[:,1:]  
    },
))

dataset=dataset.cache()
dataset=dataset.shuffle(BUFFER_SIZE)
dataset=dataset.batch(BATCH_SIZE)
dataset=dataset.prefetch(tf.data.experimental.AUTOTUNE)

<br>  
<br>  
<br>  



## 🔶Step 4. 모델 구성하기

--- 
트랜스포머 모델을 구현해보자!

In [16]:
# 패딩마스크 구현
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [17]:
# 룩어헤드 마스크
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [18]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [19]:
# 인코더 층을 쌓아 인코더 만들기
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [20]:
# 디코더 층
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재한다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [21]:
# 디코더 층을 쌓아 디코더 만들기
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [22]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [23]:
# 멀티 헤드 어텐션
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용
    query = self.query_dense(query) 
    key = self.key_dense(key) 
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만들기
    query = self.split_heads(query, batch_size)  
    key = self.split_heads(key, batch_size)  
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용
    outputs = self.dense(concat_attention)

    return outputs

In [24]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output


In [25]:
# 커스텀 학습률 스케줄러를 통한 아담 옵티마이저를 사용
# 커스텀 된 학습률
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [26]:
# 손실 함수
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [27]:
# 트랜스포머 함수를 정의
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [28]:
tf.keras.backend.clear_session()

#Hyper-parameters
D_MODEL=256 # 인코더와 디코더의 층의 개수
NUM_LAYERS=2 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS=8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS=512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT=0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3148288     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [29]:
# 모델 컴파일

learning_rate=CustomSchedule(D_MODEL)
optimizer=tf.keras.optimizers.Adam(learning_rate,beta_1=0.9,beta_2=0.98,epsilon=1e-9)

def accuracy(y_true,y_pred):
    #레이블의 크기는 (batch_size, MAX_LENGTH-1)
    y_true=tf.reshape(y_true,shape=(-1,MAX_LENGTH-1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true,y_pred)

model.compile(optimizer=optimizer,loss=loss_function,metrics=[accuracy])

In [30]:
# 모델 학습
EPOCHS=50
model.fit(dataset,epochs=EPOCHS)

Epoch 1/50
185/185 [==============================] - 16s 52ms/step - loss: 1.4496 - accuracy: 0.0202
Epoch 2/50
185/185 [==============================] - 10s 52ms/step - loss: 1.1763 - accuracy: 0.0492
Epoch 3/50
185/185 [==============================] - 10s 52ms/step - loss: 1.0029 - accuracy: 0.0508
Epoch 4/50
185/185 [==============================] - 10s 52ms/step - loss: 0.9274 - accuracy: 0.0546
Epoch 5/50
185/185 [==============================] - 10s 53ms/step - loss: 0.8706 - accuracy: 0.0575
Epoch 6/50
185/185 [==============================] - 10s 53ms/step - loss: 0.8131 - accuracy: 0.0613
Epoch 7/50
185/185 [==============================] - 10s 53ms/step - loss: 0.7490 - accuracy: 0.0670
Epoch 8/50
185/185 [==============================] - 10s 53ms/step - loss: 0.6770 - accuracy: 0.0746
Epoch 9/50
185/185 [==============================] - 10s 54ms/step - loss: 0.5981 - accuracy: 0.0835
Epoch 10/50
185/185 [==============================] - 10s 54ms/step - loss: 0.515

<br>  
<br>  
<br>  



## 🔶Step 5. 모델 평가하기

--- 
Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만들어보자

In [38]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [39]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장.
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가.
    # 이 output_sequence는 다시 디코더의 입력.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)

In [40]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

---

<br>  
<br>  

### 💥챗봇 실행해보기 

In [41]:
sentence_generation('짝녀랑 약속 파토 났어요')

입력 : 짝녀랑 약속 파토 났어요
출력 : 무슨 바쁜 일이 있었나봐요 .


'무슨 바쁜 일이 있었나봐요 .'

In [42]:
sentence_generation('헤어지고 연락했어')

입력 : 헤어지고 연락했어
출력 : 다시 헤어진 그 때로 돌아간 느낌이죠 .


'다시 헤어진 그 때로 돌아간 느낌이죠 .'

In [43]:
sentence_generation('12시 땡!')

입력 : 12시 땡!
출력 : 하루가 또 가네요 .


'하루가 또 가네요 .'

In [44]:
sentence_generation('강원도 가서 살까?')

입력 : 강원도 가서 살까?
출력 : 아름다운 곳이죠 .


'아름다운 곳이죠 .'

In [45]:
sentence_generation('갑자기 눈물 나')

입력 : 갑자기 눈물 나
출력 : 마음이 아픈가요 .


'마음이 아픈가요 .'

In [46]:
sentence_generation('후.ㅎㅎ')

입력 : 후.ㅎㅎ
출력 : 지쳤나봐요 .


'지쳤나봐요 .'

In [47]:
sentence_generation('방학이 언제야?')

입력 : 방학이 언제야?
출력 : 학생이 아니니 어쩔 수 없죠 .


'학생이 아니니 어쩔 수 없죠 .'

**맞는 answer이 나왔다.**

---

<br>  
<br>  

## ⛓️회고
<br>  

- **Keep**  
    구글링을 열심히 했다. 이 전에 했던 튜토리얼 노드를 활용했다.  
 
 <br>  
 
- **Problem**  
    함수를 작성하는 부분을 완벽히 이해하지 못했다... 어렵다..😓  
  
  <br>  
  
- **Try**  
    하나하나 보면서 더 열심히 찾아봐야겠다.  